<p style="float:right">
<img src="images/cu.png" style="display:inline" />
<img src="images/cires.png" style="display:inline" />
<img src="images/nasa.png" style="display:inline" />
</p>

# Python, Jupyter & pandas tutorial: Module 2

## Obtaining data and basic inspection

### Basic data access

- It is, of course, possible to obtain data (rougly construed -- we'll look at images here because they're simple to view) externally (or via the `%%script` magic, which saves the trouble of opening a separate terminal / command / browser window). We can fetch an image to the local filesystem, then display it with Markdown:

In [ ]:
%%script bash
wget ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/Feb/N_197902_conc.png

<img src='N_197902_conc.png' style='float:left'/>

- Or, we can obtain an image directly from the internet and display in with Python code:

In [ ]:
from IPython.display import Image
Image(url='ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/Feb/N_201602_conc.png')

# Yo! Do the "Total area" figures account for the difference in pole-hole size? Seems like, here, the hole is full of ice, but what about earlier/later in the season?

### OpenDAP data access

